In [1]:
from keys import odds_api_key
import pandas as pd
import requests
import json

In [2]:
url = "https://api.the-odds-api.com/v3/sports/"

params = {
    'apiKey': odds_api_key,
}

In [3]:
sport_response = requests.get(url,params).json()

In [4]:
#Example of the sports call
print(json.dumps(sport_response,indent = 4,sort_keys=True))

{
    "data": [
        {
            "active": true,
            "details": "US College Football",
            "group": "American Football",
            "key": "americanfootball_ncaaf",
            "title": "NCAAF"
        },
        {
            "active": true,
            "details": "Aussie Football",
            "group": "Aussie Rules",
            "key": "aussierules_afl",
            "title": "AFL"
        },
        {
            "active": true,
            "details": "Mixed Martial Arts",
            "group": "Mixed Martial Arts",
            "key": "mma_mixed_martial_arts",
            "title": "MMA"
        },
        {
            "active": true,
            "details": "Aussie Rugby League",
            "group": "Rugby League",
            "key": "rugbyleague_nrl",
            "title": "NRL"
        },
        {
            "active": true,
            "details": "Aussie Soccer \ud83c\udde6\ud83c\uddfa",
            "group": "Soccer - Other",
            "key": "soccer_austr

In [5]:
#Grab all of the keys for active sports
sport_keys = [[sport['key'],sport['title']] for sport in sport_response['data']]
sport_keys

[['americanfootball_ncaaf', 'NCAAF'],
 ['aussierules_afl', 'AFL'],
 ['mma_mixed_martial_arts', 'MMA'],
 ['rugbyleague_nrl', 'NRL'],
 ['soccer_australia_aleague', 'A-League'],
 ['soccer_sweden_allsvenskan', 'Allsvenskan - Sweden'],
 ['soccer_sweden_superettan', 'Superettan - Sweden'],
 ['soccer_turkey_super_league', 'Turkey Super League']]

In [6]:
# Valid regions are au (Australia), uk (United Kingdom), eu (Europe) and us (United States)
valid_regions = ['au','uk','eu','us']

In [7]:
#Reset URL to grab the odds rather than sports
url = "https://api.the-odds-api.com/v3/odds/?"

In [8]:
#Example of 
af_response = requests.get('https://api.the-odds-api.com/v3/odds/?apiKey=6fe97a3c01dd76520fa61094f19bbf82&sport=soccer_turkey_super_league&region=eu').json()
print(json.dumps(af_response['data'][0],indent=4,sort_keys=True))

{
    "commence_time": 1584723600,
    "home_team": "Fenerbahce",
    "sites": [
        {
            "last_update": 1584581474,
            "odds": {
                "h2h": [
                    1.32,
                    8.0,
                    5.5
                ]
            },
            "site_key": "unibet",
            "site_nice": "Unibet"
        },
        {
            "last_update": 1584581514,
            "odds": {
                "h2h": [
                    1.32,
                    8.59,
                    5.91
                ]
            },
            "site_key": "pinnacle",
            "site_nice": "Pinnacle"
        },
        {
            "last_update": 1584581623,
            "odds": {
                "h2h": [
                    1.33,
                    10.0,
                    6.0
                ],
                "h2h_lay": [
                    1.34,
                    11.0,
                    6.6
                ]
            },
            "site_

In [9]:
#Defining the output lists for the dataframe
leagues = []
regions = []
home_teams = []
away_teams = []
commence_times = []
last_updates = []
h2h_odds = []
h2h_lay_odds = []

In [10]:
for sport, league in sport_keys:
    params['sport'] = sport
    for region in valid_regions:
        params['region'] = region
        response = requests.get(url,params).json()
        games = response['data']
        for game in games:
            for site in game['sites']:
                if site['site_nice'] == 'Betfair':
                    leagues.append(game['sport_nice'])
                    regions.append(region)
                    home_teams.append(game['teams'][1])
                    away_teams.append(game['teams'][0])
                    commence_times.append(game['commence_time'])
                    last_updates.append(site['last_update'])
                    h2h_odds.append(site['odds']['h2h'])
                    h2h_lay_odds.append(site['odds']['h2h_lay'])

In [11]:
df = pd.DataFrame({'League':leagues,
                   'Region':regions,
                   'Home Team':home_teams,
                   'Away Team':away_teams,
                   'Commence Times':commence_times,
                   'Last Updated':last_updates,
                   'H2H Odds':h2h_odds,
                   'H2H Lay Odds':h2h_lay_odds})
df

,League,Region,Home Team,Away Team,Commence Times,Last Updated,H2H Odds,H2H Lay Odds
0,AFL,au,Richmond Tigers,Carlton Blues,1584606300,1584581548,"[4.1, 1.31]","[4.3, 1.32]"
1,AFL,au,Western Bulldogs,Collingwood Magpies,1584694200,1584581548,"[1.86, 2.08]","[1.92, 2.18]"
2,AFL,au,Fremantle Dockers,Essendon Bombers,1584758700,1584581548,"[1.52, 2.82]","[1.54, 2.98]"
3,AFL,au,Sydney Swans,Adelaide Crows,1584768900,1584581548,"[1.75, 2.18]","[1.84, 2.34]"
4,AFL,au,Greater Western Sydney Giants,Geelong Cats,1584779100,1584581548,"[2.52, 1.62]","[2.62, 1.66]"
...,...,...,...,...,...,...,...,...
83,Turkey Super League,us,Besiktas JK,Antalyaspor,1584810000,1584581623,"[6.8, 1.55, 4.4]","[7.8, 1.56, 4.9]"
84,Turkey Super League,us,Torku Konyaspor,Genclerbirligi SK,1584873000,1584581623,"[2.36, 3.35, 3.3]","[2.48, 3.7, 3.5]"
85,Turkey Super League,us,Çaykur Rizespor,Galatasaray,1584882000,1584581623,"[1.76, 4.8, 4.0]","[1.85, 5.5, 4.2]"
86,Turkey Super League,us,Basaksehir,Alanyaspor,1584892800,1584581623,"[4.5, 1.9, 3.55]","[5.3, 1.95, 4.0]"


In [12]:
df.to_csv("OddsAPI_data.csv")